In [4]:
library(pacman)
p_load(tidyverse, data.table, ggpubr, scales, remotes, ieugwasr, genetics.binaRies)
setwd("../scripts")
p_load_gh("MRCIEU/TwoSampleMR")
#if (!requireNamespace("remotes", quietly = TRUE)) install.packages("remotes")
#remotes::install_github("MRCIEU/genetics.binaRies") (works for R version.4.2.2)

### Functions

In [5]:
## read Brain Cortex tissue GRN
read_tissue_map <- function() {
    dir <- "../../../../tissue_maps/Brain_Cortex/"
    res <- tibble()
    cortex_map <- file.path(list.files(dir, 
                            pattern = "significant_eqtls.txt", 
                            recursive = TRUE,
                            full.names = TRUE))
    res <- res %>%
            bind_rows(
                read_tsv(cortex_map, show_col_types = FALSE)) %>% 
            dplyr::select(snp, beta, beta_se, alt, ref, maf, eqtl_pval, 
                          gene, interaction_type) %>% 
            distinct()
} 

## Prepare data for MR
## 1. Prepare exposure data
prepare_exposure_data <- function(eqtl_file, fp){
    if (!dir.exists(fp)){
        dir.create(fp)
    }
    exposure_df <- read_exposure_data(
                        filename = file.path(eqtl_file),
                        sep = "\t",
                        snp_col = "snp",
                        beta_col = "beta",
                        se_col = "beta_se",
                        effect_allele_col = "alt",
                        other_allele_col = "ref",
                        eaf_col = "maf",
                        pval_col = "eqtl_pval",
                        phenotype_col = "gene") %>% 
                   distinct() %>% 
                   write_tsv(file.path(fp, "exposure_df.txt"))
}

## 2. Clump SNPs in the exposure dataset
clump_snps <- function(exposure_df, fp) {
    res <- tibble()
    for (id in unique(exposure_df$id.exposure)){
        exp_data <- exposure_df %>%
            dplyr::select(SNP, pval.exposure, id.exposure) %>% 
            distinct() %>% 
            filter(id.exposure == id) %>% 
            rename("rsid" = "SNP",
                   "pval" = "pval.exposure",
                   "id" = "id.exposure")
        res <- res %>% 
        bind_rows(tryCatch({
            ieugwasr::ld_clump(
                exp_data,
                bfile = "../data/EUR/EUR",
                plink_bin = genetics.binaRies::get_plink_binary()
            )}, error = function(e){}))
    }
    res <- res %>% 
        rename("SNP" = "rsid",
               "pval.exposure" = "pval",
               "id.exposure" = "id") %>%
        inner_join(exposure_df, 
                   by = c("SNP", "pval.exposure", "id.exposure")) %>% 
        write_tsv(file.path(fp, "exposure_ld_clumped_df.txt"))
}

## 2. Extract outcome data
extract_outcome <- function(exposure_df, gwas, fp){
    outcome_df <- extract_outcome_data(
                        snps = exposure_df$SNP,
                        outcomes = gwas,
                        proxies = TRUE,
                        rsq = 0.9) %>%  
                  distinct() %>% 
                  write_tsv(file.path(fp, "outcome_df.txt"))
}    

## 3. Harmonize exposure outcome effects
harmonise_expo_outcome <- function(exposure_df, outcome_df, fp) {
    harmonised_df <- harmonise_data(
                        exposure_dat = exposure_df, 
                        outcome_dat = outcome_df) %>% 
                     filter(!mr_keep == "FALSE") %>% # Remove SNPs that failed harmonisation
                     distinct() %>%
                     write_tsv(file.path(fp, "harmonised_eo_df.txt")) # using only one gwas study for outcome so power pruning is not required 
}

## Performs the analysis multiple times for each exposure-outcome combination - each time using a different single SNP to perform the analysis

perform_singlesnp_mr_single <- function(df, method = NULL) {
    if (!is.null(method)) {
        res <- mr_singlesnp(df, single_method = method)
    }
    else {
        res <- mr_singlesnp(df)
    }            
}

create_file_ifnot_exists <- function(out_fp, grn){
    #grn <- str_replace_all(grn, c("'" = "", " " = "_"))
    if (!dir.exists(out_fp)){
        dir.create(out_fp)
    }
    fp <- file.path(out_fp, paste0(grn, "_data_for_exposure.txt"))
    return(fp)
}


### Prepare exposure data

In [ ]:
cortex_grn_eqtl_info <- read_tissue_map() %>% 
                dplyr::select(snp, beta, beta_se, alt, ref, maf, eqtl_pval, gene) %>% 
                filter(eqtl_pval < 1*10^-5) %>% 
                distinct() %>% 
                write_tsv(create_file_ifnot_exists("../analysis/bcgrn_MR", "bcgrn"))

out_file_path <- '../analysis/bcgrn_MR'
exposure_data <- prepare_exposure_data(
                    '../analysis/bcgrn_MR/bcgrn_data_for_exposure.txt',
                    out_file_path
                    )

### Clump exposure SNPs

In [ ]:
out_file_path <- '../analysis/bcgrn_MR'
exposure_data <- read_tsv('../analysis/bcgrn_MR/bcgrn_data_for_exposure.txt')
clump_expo_snps <- clump_snps(
                        exposure_data,
                        out_file_path
                        )

### Extract outcome data

In [ ]:
out_file_path <- '../analysis/bcgrn_MR'
outcome_data <- extract_outcome(exposure_clumped, 'ieu-b-7', out_file_path)

### Harmonise exposure/outcome data

In [ ]:
out_file_path <- '../analysis/bcgrn_MR'
harmonised_df <- harmonise_expo_outcome(exposure_clumped, outcome_data, out_file_path)

#### MR input data

In [ ]:
mr_input_data <- read_tsv("../analysis/bcgrn_MR/harmonised_eo_df.txt",
                         show_col_types = FALSE) 

#### Separate exposure with single and multiple instruments

In [ ]:
# exposure with single SNP instrument
single_snp_exposures <- mr_input_data %>% 
                group_by(exposure) %>% 
                summarise(instruments = length(SNP)) %>% 
                filter(instruments == 1)

# exposure with multiple SNP instruments
inp_genes_many_snp <- mr_input_data %>% 
                group_by(exposure) %>% 
                summarise(instruments = length(SNP)) %>% 
                filter(instruments > 1)

#### Perform sensitivity analysis for exposures with multiple snp instruments

In [ ]:
## do sensitivity analysis (remove Q_pval < 0.05)
hetero_test_passed <- mr_heterogeneity(mr_input_data %>% 
                                filter(exposure %in% inp_genes_many_snp$exposure) %>% 
                                distinct()) %>% 
                        filter(!Q_pval < 0.05)

## do horizontal pleiotropy test (remove p-val > 0.05 or NA)
pleiotropy_test_passed <- mr_pleiotropy_test(mr_input_data %>% 
                                filter(exposure %in% inp_genes_many_snp$exposure) %>% 
                                distinct()) %>% 
                         filter(!pval == "NA" | pval > 0.05) 

### Perform MR for exposure with single instrument

In [ ]:
mr_input_data_wald <- mr_input_data %>% 
                        filter(exposure %in% single_snp_exposures$exposure) %>% 
                        distinct()
 
wald_mr_res <- perform_singlesnp_mr_single(mr_input_data_wald, "mr_wald_ratio") %>% 
                write_tsv("../analysis/bcgrn_MR/grn_full_mr_res_wald.txt")

#### Generate odds ratios for MR results

In [ ]:
wald_mr_res <- read_tsv("../analysis/bcgrn_MR/grn_full_mr_res_wald.txt",
                       show_col_types = FALSE)

## Perform MR strong effects
wald_mr_res_strong <- wald_mr_res %>% 
                        filter(!is.na(p)) %>% 
                        generate_odds_ratios() %>% 
                        filter(p < 0.05/n_distinct(exposure)) %>% 
                        write_tsv("../analysis/bcgrn_MR/cortex_grn_singlesnp_strong.txt")

## Perform MR suggestive effects
wald_mr_res_sugg <- wald_mr_res %>% 
                        filter(!is.na(p)) %>%
                        generate_odds_ratios() %>%
                        filter(p > 0.05/n_distinct(exposure) & p < 0.05) %>% 
                        write_tsv("../analysis/bcgrn_MR/cortex_grn_singlesnp_sugg.txt")

#### Perform ivw (inverse variance weighted) MR analysis

In [ ]:
## perform MR for exposure with multiple instruments
mr_input_data_ivw <- mr_input_data %>% 
                        filter(exposure %in% hetero_test_passed$exposure) %>% 
                        distinct() 

mr_res_ivr <- perform_singlesnp_mr_single(mr_input_data_ivw, "mr_ivw") %>% 
                        filter(!is.na(p) & !SNP == "All - MR Egger") %>%
                        generate_odds_ratios() %>%
                        filter(p < 0.05/n_distinct(exposure)) #no significant results